# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GraphRepresentationGenerator** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from graph_representation_generator import GraphRepresentationGenerator
from dataset_manager import (
    MovieLensManager,
    PROMPT_KGE_DIMENSION,
    INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    ROOT,
)
from llm_manager import (
    PromptBertClassifier,
    VanillaBertClassifier,
    InputEmbedsReplaceClassifier,
)

In [8]:
EPOCHS = 20
BATCH_SIZE_KGE = 128000
BATCH_SIZE_LLM = 256

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [3]:
kg_manager = MovieLensManager()

In [4]:
kg_manager.llm_df

,source_id,target_id,id_x,id_y,prompt_feature_title,prompt_feature_genres,labels,split,split_x,split_y,_merge
0,0,16,0,16,Sense and Sensibility (1995),"['Drama', 'Romance']",1,train,NaN,NaN,NaN
1,0,16,0,16,Sense and Sensibility (1995),"['Drama', 'Romance']",1,val,NaN,NaN,NaN
2,0,24,0,24,Leaving Las Vegas (1995),"['Drama', 'Romance']",1,train,NaN,NaN,NaN
3,0,24,0,24,Leaving Las Vegas (1995),"['Drama', 'Romance']",1,test,NaN,NaN,NaN
4,0,28,0,28,"City of Lost Children, The (Cité des enfants p...","['Adventure', 'Drama', 'Fantasy', 'Mystery', '...",1,val,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
46720294,200947,15212,200947,15212,Vampires Suck (2010),['Comedy'],1,test,NaN,NaN,NaN
46720295,200947,15240,200947,15240,"Social Network, The (2010)",['Drama'],1,train,NaN,NaN,NaN
46720296,200947,15240,200947,15240,"Social Network, The (2010)",['Drama'],1,test,NaN,NaN,NaN
46720297,200947,16546,200947,16546,Beginners (2010),['Drama'],1,train,NaN,NaN,NaN


First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-32m.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [5]:
kg_manager.data

HeteroData(
  source={ node_id=[200948] },
  target={
    node_id=[87585],
    x=[87585, 20],
  },
  (source, edge, target)={ edge_index=[2, 32000204] },
  (target, rev_edge, source)={ edge_index=[2, 32000204] }
)

Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [9]:
graph_representation_generator_prompt = GraphRepresentationGenerator(
    kg_manager.data,
    kg_manager.gnn_train_data,
    kg_manager.gnn_val_data,
    kg_manager.gnn_test_data,
    kge_dimension=PROMPT_KGE_DIMENSION,
    force_recompute=True,
)
graph_representation_generator_input_embeds_replace = GraphRepresentationGenerator(
    kg_manager.data,
    kg_manager.gnn_train_data,
    kg_manager.gnn_val_data,
    kg_manager.gnn_test_data,
    hidden_channels=INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    kge_dimension=INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    force_recompute=True,
)

Device: 'cpu'
Device: 'cpu'


We then train and validate the model on the link prediction task.

If the model is already trained, we can skip this part.
Training the models can take up to 5 minutes.

In [10]:
print("Prompt Training")
graph_representation_generator_prompt.train_model(
    kg_manager.gnn_train_data, EPOCHS, BATCH_SIZE_KGE
)
graph_representation_generator_prompt.validate_model(
    kg_manager.gnn_test_data, batch_size=BATCH_SIZE_KGE
)
print("Attention Training")
graph_representation_generator_input_embeds_replace.train_model(
    kg_manager.gnn_train_data, EPOCHS, BATCH_SIZE_KGE
)
graph_representation_generator_input_embeds_replace.validate_model(
    kg_manager.gnn_test_data, batch_size=BATCH_SIZE_KGE
)


Prompt Training


100%|██████████| 61/61 [07:58<00:00,  7.85s/it]


Epoch: 001, Loss: 0.4455


100%|██████████| 61/61 [07:57<00:00,  7.83s/it]


Epoch: 002, Loss: 0.2166


100%|██████████| 61/61 [08:00<00:00,  7.88s/it]


Epoch: 003, Loss: 0.1780


 70%|███████   | 43/61 [05:49<02:26,  8.11s/it]

Next we produce the KGEs for every edge in the dataset. These embeddings can then be used for the LLM on the link-prediction task.

In [6]:
prompt_embeddings = graph_representation_generator_prompt.get_saved_embeddings("prompt")
input_embeds_replace_embeddings = (
    graph_representation_generator_input_embeds_replace.get_saved_embeddings(
        "input_embeds_replace"
    )
)
save_prompt = False
save_input_embeds_replace = False
if prompt_embeddings is None:
    prompt_embeddings = graph_representation_generator_prompt.generate_embeddings(
        kg_manager.llm_df
    )
    save_prompt = True
if input_embeds_replace_embeddings is None:
    input_embeds_replace_embeddings = (
        graph_representation_generator_input_embeds_replace.generate_embeddings(
            kg_manager.llm_df
        )
    )
    save_input_embeds_replace = True

kg_manager.append_prompt_graph_embeddings(prompt_embeddings, save=save_prompt)
kg_manager.append_input_embeds_replace_graph_embeddings(
    input_embeds_replace_embeddings, save=save_input_embeds_replace
)


In [ ]:
df = kg_manager.add_false_edges(
    1.0,
    graph_representation_generator_prompt.get_embedding,
    graph_representation_generator_input_embeds_replace.get_embedding,
    splits=["val", "test"],
)
df[df["split"] == "val"]

Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [7]:
VANILLA_ROOT = f"{ROOT}/llm/vanilla"
PROMPT_ROOT = f"{ROOT}/llm/prompt"
INPUT_EMBEDS_REPLACE_ROOT = f"{ROOT}/llm/input_embeds_replace"

In [ ]:
vanilla_bert_classifier = VanillaBertClassifier(
    kg_manager.llm_df,
    kg_manager.source_df,
    kg_manager.target_df,
    root_path=VANILLA_ROOT,
)

Next we generate a vanilla llm dataset and tokenize it for training.

In [ ]:
dataset_vanilla = kg_manager.generate_vanilla_dataset(
    vanilla_bert_classifier.tokenize_function
)

Next we train the model on the produced dataset. This can be skipped, if already trained ones.

In [ ]:
vanilla_bert_classifier.train_model_on_data(
    dataset_vanilla, epochs=EPOCHS, batch_size=BATCH_SIZE
)

Next we initialize the prompt encoder only classifier. This classifier uses the vanilla prompt and the KGEs for its link prediction.

In [ ]:
prompt_bert_classifier = PromptBertClassifier(
    kg_manager,
    graph_representation_generator_prompt.get_embedding,
    model_max_length=512,
    root_path=PROMPT_ROOT,
)

We also generate a prompt dataset, this time the prompts also include the KGEs.

In [ ]:
dataset_prompt = kg_manager.generate_prompt_embedding_dataset(
    prompt_bert_classifier.tokenize_function,
)

We also train the model. This can be skipped if already done ones.

In [ ]:
prompt_bert_classifier.train_model_on_data(
    dataset_prompt, epochs=EPOCHS, batch_size=BATCH_SIZE
)

In [ ]:
input_embeds_replace_bert_classifier = InputEmbedsReplaceClassifier(
    kg_manager,
    graph_representation_generator_input_embeds_replace.get_embedding,
    root_path=INPUT_EMBEDS_REPLACE_ROOT,
)

In [ ]:
dataset_embedding = kg_manager.generate_input_embeds_replace_embedding_dataset(
    input_embeds_replace_bert_classifier.tokenizer.sep_token,
    input_embeds_replace_bert_classifier.tokenizer.pad_token,
    input_embeds_replace_bert_classifier.tokenize_function,
)

In [ ]:
input_embeds_replace_bert_classifier.train_model_on_data(
    dataset_embedding, epochs=EPOCHS, batch_size=BATCH_SIZE
)